In [ ]:
import os
import json
from addict import Dict
import glob
import yaml
import numpy as np
import torch
import torch.nn as nn
import torch.functional as F
from torch.utils.data import DataLoader, DistributedSampler
from torch.distributed import init_process_group, destroy_process_group
import torch.multiprocessing as mp
import timm
from dataset import TotalSegmenatatorData

In [ ]:
CONFIG_FILE = "config/config.yaml"

In [ ]:
cfgs = Dict(yaml.load(open(CONFIG_FILE), "r"), Loader=yaml.Loader)
paths = cfgs.paths
md_cfgs = cfgs.metadata_params
data_cfgs = cfgs.dataset_params
optim_cfgs = cfgs.optimizer_params
schedule_cfgs = cfgs.scheduler_params
model_cfgs = cfgs.model_params
train_cfgs = train_params
test_cfgs = test_params

## Training

## Testing